In [3]:
import gc
gc.collect()

435

In [4]:
from dask.distributed import Client

client = Client(n_workers=4)

C:\Users\dthoms\AppData\Local\Continuum\anaconda3\lib\site-packages\distributed\dashboard\core.py:79: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [1]:
import numpy as np
import pandas as pd

Dask definitely faster

In [2]:
%%time

trainData = pd.read_csv('../kaggle-data/train.csv')

Wall time: 7.48 s


In [3]:
trainData['timestamp'] = pd.to_datetime(trainData.timestamp)
trainData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
 #   Column         Dtype         
---  ------         -----         
 0   building_id    int64         
 1   meter          int64         
 2   timestamp      datetime64[ns]
 3   meter_reading  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 616.9 MB


In [4]:
trainWeatherData = pd.read_csv("../kaggle-data/weather_train.csv")

In [5]:
trainWeatherData['timestamp'] = pd.to_datetime(trainWeatherData.timestamp)
trainWeatherData['hour'] = trainWeatherData.timestamp.dt.hour
trainWeatherData['weekday'] = trainWeatherData.timestamp.dt.dayofweek
trainWeatherData['month'] = trainWeatherData.timestamp.dt.month
trainWeatherData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   site_id             139773 non-null  int64         
 1   timestamp           139773 non-null  datetime64[ns]
 2   air_temperature     139718 non-null  float64       
 3   cloud_coverage      70600 non-null   float64       
 4   dew_temperature     139660 non-null  float64       
 5   precip_depth_1_hr   89484 non-null   float64       
 6   sea_level_pressure  129155 non-null  float64       
 7   wind_direction      133505 non-null  float64       
 8   wind_speed          139469 non-null  float64       
 9   hour                139773 non-null  int64         
 10  weekday             139773 non-null  int64         
 11  month               139773 non-null  int64         
dtypes: datetime64[ns](1), float64(7), int64(4)
memory usage: 12.8 MB


In [6]:
bMetaData = pd.read_csv("../kaggle-data/building_metadata.csv")
bMetaData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   site_id      1449 non-null   int64  
 1   building_id  1449 non-null   int64  
 2   primary_use  1449 non-null   object 
 3   square_feet  1449 non-null   int64  
 4   year_built   675 non-null    float64
 5   floor_count  355 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 68.0+ KB


In [7]:
%%time
combDF = pd.merge(bMetaData,trainData,how='inner',on='building_id')
combDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20216100 entries, 0 to 20216099
Data columns (total 9 columns):
 #   Column         Dtype         
---  ------         -----         
 0   site_id        int64         
 1   building_id    int64         
 2   primary_use    object        
 3   square_feet    int64         
 4   year_built     float64       
 5   floor_count    float64       
 6   meter          int64         
 7   timestamp      datetime64[ns]
 8   meter_reading  float64       
dtypes: datetime64[ns](1), float64(3), int64(4), object(1)
memory usage: 1.5+ GB
Wall time: 6.07 s


In [8]:
del trainData
import gc
gc.collect()

93

In [9]:
%%time
combDF = pd.merge(combDF,trainWeatherData,how='inner',on=['site_id','timestamp'])

Wall time: 6.62 s


In [10]:
combDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20125605 entries, 0 to 20125604
Data columns (total 19 columns):
 #   Column              Dtype         
---  ------              -----         
 0   site_id             int64         
 1   building_id         int64         
 2   primary_use         object        
 3   square_feet         int64         
 4   year_built          float64       
 5   floor_count         float64       
 6   meter               int64         
 7   timestamp           datetime64[ns]
 8   meter_reading       float64       
 9   air_temperature     float64       
 10  cloud_coverage      float64       
 11  dew_temperature     float64       
 12  precip_depth_1_hr   float64       
 13  sea_level_pressure  float64       
 14  wind_direction      float64       
 15  wind_speed          float64       
 16  hour                int64         
 17  weekday             int64         
 18  month               int64         
dtypes: datetime64[ns](1), float64(10), int64

In [2]:
gc.get_objects()

[<weakref at 0x000002AF15521F98; dead>,
 ('INSERT INTO history VALUES (?, ?, ?, ?)',),
 {'_trait_values': {'enabled': True,
   'format_type': 'text/plain',
   'print_method': '_ipython_display_',
   'parent': <IPython.core.formatters.DisplayFormatter at 0x2af154f4888>,
   'config': {'IPKernelApp': {'connection_file': 'C:\\Users\\dthoms\\AppData\\Roaming\\jupyter\\runtime\\kernel-67e49dcb-be2c-4489-83c5-b695146c7956.json'}},
   'singleton_printers': {},
   'type_printers': {},
   'deferred_printers': {}},
  '_trait_notifiers': {'config': {'change': [<traitlets.traitlets.ObserveHandler at 0x2af13457d48>]}},
  '_trait_validators': {},
  '_cross_validation_lock': False},
 {'change': [<traitlets.traitlets.ObserveHandler at 0x2af13457d48>]},
 {'config': {'change': [<traitlets.traitlets.ObserveHandler at 0x2af13457d48>]}},
 {'enabled': True,
  'format_type': 'text/plain',
  'print_method': '_ipython_display_',
  'parent': <IPython.core.formatters.DisplayFormatter at 0x2af154f4888>,
  'config'

In [179]:
trainWeatherData_dd = dd.from_pandas(trainWeatherData,npartitions=4)
type(trainWeatherData_dd)

dask.dataframe.core.DataFrame

In [180]:
bMetaData_dd = dd.from_pandas(bMetaData,npartitions=100)
type(bMetaData_dd)

dask.dataframe.core.DataFrame

In [181]:
trainData_dd = dd.from_pandas(trainData,npartitions=100)

In [182]:
type(trainData_dd)

dask.dataframe.core.DataFrame

In [184]:
combDF.dtypes

site_id                   int64
building_id               int64
primary_use              object
square_feet               int64
year_built              float64
floor_count             float64
meter                     int64
timestamp        datetime64[ns]
meter_reading           float64
dtype: object

In [187]:
%%time
combDF.compute()
type(CombDF)

MemoryError: 

In [150]:
combDF

,site_id,building_id,primary_use,square_feet,year_built,floor_count,meter,timestamp,meter_reading,timespace
npartitions=100,,,,,,,,,,
,int64,int64,object,int64,float64,float64,int64,object,float64,datetime64[ns]
,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


In [6]:
%%time
df = pd.read_csv('../kaggle-data/train.csv')
df.info()

Wall time: 8.66 s


# Check original code and see if can implement dask

In [53]:
bMetaData = dd.from_pandas(pd.read_csv("../kaggle-data/building_metadata.csv"),npartitions=100).compute()
trainData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
 #   Column         Dtype         
---  ------         -----         
 0   building_id    int64         
 1   meter          int64         
 2   timestamp      datetime64[ns]
 3   meter_reading  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 616.9 MB


In [54]:
trainData = dd.from_pandas(pd.read_csv("../kaggle-data/train.csv"),npartitions=10).compute()
trainData.tail()

,building_id,meter,timestamp,meter_reading
20216095,1444,0,2016-12-31 23:00:00,8.750
20216096,1445,0,2016-12-31 23:00:00,4.825
20216097,1446,0,2016-12-31 23:00:00,0.000
20216098,1447,0,2016-12-31 23:00:00,159.575
20216099,1448,0,2016-12-31 23:00:00,2.850


In [55]:
trainWeatherData = dd.from_pandas(pd.read_csv("../kaggle-data/weather_train.csv"),npartitions=100).compute()
trainWeatherData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   site_id             139773 non-null  int64  
 1   timestamp           139773 non-null  object 
 2   air_temperature     139718 non-null  float64
 3   cloud_coverage      70600 non-null   float64
 4   dew_temperature     139660 non-null  float64
 5   precip_depth_1_hr   89484 non-null   float64
 6   sea_level_pressure  129155 non-null  float64
 7   wind_direction      133505 non-null  float64
 8   wind_speed          139469 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 9.6+ MB


In [56]:
#permutations = dd.from_array(np.random.permutation(len(df)))
#df['rand_index'] = permutations

#breakdown timestamp

trainData['timestamp'] = dd.to_datetime(trainData.timestamp).compute()

trainData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
 #   Column         Dtype         
---  ------         -----         
 0   building_id    int64         
 1   meter          int64         
 2   timestamp      datetime64[ns]
 3   meter_reading  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 616.9 MB


In [61]:
trainWeatherData['timestamp'] = dd.to_datetime(trainWeatherData.timestamp).compute()
trainWeatherData['hour'] = trainWeatherData.timestamp.dt.hour
trainWeatherData['weekday'] = trainWeatherData.timestamp.dt.dayofweek
trainWeatherData['month'] = trainWeatherData.timestamp.dt.month
trainWeatherData.tail()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,weekday,month
139768,15,2016-12-31 19:00:00,3.0,NaN,-8.0,NaN,NaN,180.0,5.7,19,5,12
139769,15,2016-12-31 20:00:00,2.8,2.0,-8.9,NaN,1007.4,180.0,7.7,20,5,12
139770,15,2016-12-31 21:00:00,2.8,NaN,-7.2,NaN,1007.5,180.0,5.1,21,5,12
139771,15,2016-12-31 22:00:00,2.2,NaN,-6.7,NaN,1008.0,170.0,4.6,22,5,12
139772,15,2016-12-31 23:00:00,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8,23,5,12


In [62]:
#dmap metering points, day of week
#weekdayName = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
#trainWeatherData['weekday'] = trainWeatherData['weekday'].map(weekdayName)
#monthName = {1:'Jan',2:'Feb',3:'Mar',4:'April',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
#trainWeatherData['month'] = trainWeatherData['month'].map(monthName)
#meterName = {0: 'electricity', 1: 'chilledwater', 2: 'steam', 3: 'hotwater'}
#trainData['meter'] = trainData['meter'].map(meterName)

#combine buildingMetaData and trainData
combDF = dd.merge(bMetaData,trainData,how='inner',on='building_id').compute()
combDF.tail()

AttributeError: 'DataFrame' object has no attribute 'compute'

In [63]:
%%time
#combine combDF with revised trainWeatherData
combDF = dd.merge(combDF,trainWeatherData,how='inner',on=['site_id','timestamp'])
combDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20125605 entries, 0 to 20125604
Data columns (total 19 columns):
 #   Column              Dtype         
---  ------              -----         
 0   site_id             int64         
 1   building_id         int64         
 2   primary_use         object        
 3   square_feet         int64         
 4   year_built          float64       
 5   floor_count         float64       
 6   meter               int64         
 7   timestamp           datetime64[ns]
 8   meter_reading       float64       
 9   air_temperature     float64       
 10  cloud_coverage      float64       
 11  dew_temperature     float64       
 12  precip_depth_1_hr   float64       
 13  sea_level_pressure  float64       
 14  wind_direction      float64       
 15  wind_speed          float64       
 16  hour                int64         
 17  weekday             int64         
 18  month               int64         
dtypes: datetime64[ns](1), float64(10), int64

In [64]:
combDF = combDF.drop('year_built',axis=1)
df = combDF.compute()
# dask operation
#raw_data = raw_data.drop('some_great_column', axis=1)

# conversion to pandas
#df = raw_data.compute()

AttributeError: 'DataFrame' object has no attribute 'compute'

In [ ]:
combDF.dropna(inplace = True)

In [ ]:
#remove following meters because of bad data
combDF.drop(combDF[(combDF['building_id'] == 1099) & (combDF['meter'] == 'steam')].index,inplace=True)
combDF.drop(combDF[(combDF['building_id'] == 778) & (combDF['meter'] == 'chilledwater')].index,inplace=True)
combDF.drop(combDF[(combDF['building_id'] == 1021) & (combDF['meter'] == 'hotwater')].index,inplace=True)
combDF.drop(combDF[(combDF['building_id'] == 1168) & (combDF['meter'] == 'steam')].index,inplace=True)
combDF.drop(combDF[(combDF['building_id'] == 1197) & (combDF['meter'] == 'steam')].index,inplace=True)
combDF.drop(combDF[(combDF['building_id'] == 1148) & (combDF['meter'] == 'steam')].index,inplace=True)
combDF.drop(combDF[(combDF['building_id'] == 1159) & (combDF['meter'] == 'steam')].index,inplace=True)


combDF = combDF[['timestamp','hour','weekday','month', 'site_id', 'building_id', 'meter', 'meter_reading', 'primary_use', 'square_feet', 'air_temperature', 'dew_temperature', 'sea_level_pressure', 'wind_direction', 'wind_speed']]


#output csv
combDF.to_csv("data/combDF.csv", index = False)
# trainWeatherData.to_csv("data/trainWeatherData.csv", index = False)
# trainData.to_csv("data/trainData.csv", index = False)
# bMetaData.to_csv("data/bMetaData.csv", index = False)

#create samples
bMDSample = bMetaData.head(100)
trainSample = trainData.head(100)
weathTrainSample = trainWeatherData.head(100)
combDFSample = combDF.head(100)

In [122]:
df = pd.DataFrame(dict(a=list('aabbcc'), b=list(range(6))), index=pd.date_range(start='20100101', periods=6))
type(df)
#ddf = dd.from_pandas(df, npartitions=3)

pandas.core.frame.DataFrame